# Setup

In [10]:
import os
import zipfile
from pathlib import Path
from dotenv import load_dotenv


def check_env() -> str:
    if os.environ.get('KAGGLE_KERNEL_RUN_TYPE'):
        print("Running on Kaggle")
        return "kaggle"
    else:
        print("Running locally")
        return "local"


ENV = check_env()

if ENV == "kaggle":
    data_dir = Path("/kaggle/input/ka-ocr")
else:
    load_dotenv()

    from huggingface_hub import hf_hub_download

    data_dir = Path("./data")
    data_dir.mkdir(parents=True, exist_ok=True)

    hf_repo = os.getenv("HF_DATASET_REPO")
    hf_token = os.getenv("HF_TOKEN")

    if not hf_repo:
        raise ValueError("HF_DATASET_REPO not set in .env")

    # Download with automatic caching - skips if local matches remote (etag-based)
    zip_path = hf_hub_download(
        repo_id=hf_repo,
        filename="ka-ocr.zip",
        repo_type="dataset",
        token=hf_token,
        local_dir=data_dir,
    )

    # Extract only if not already extracted OR if zip is newer than extraction
    extract_marker = data_dir / ".extracted"
    zip_file = Path(zip_path)
    needs_extract = (
        not extract_marker.exists() or
        zip_file.stat().st_mtime > extract_marker.stat().st_mtime
    )

    if needs_extract:
        print("Extracting dataset...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        extract_marker.touch()
        print("Extraction complete")
    else:
        print("Dataset already extracted, skipping")

print(f"\nDataset contents in {data_dir}:")
for item in data_dir.iterdir():
    if not item.name.startswith('.') and item.name != "ka-ocr.zip":
        print(f"  {item.name}")

Running locally
Dataset already extracted, skipping

Dataset contents in data:
  3d_unicode
  alkroundedmtav-medium
  alkroundednusx-medium
  ar-archy-regular
  arial_geo
  arial_geo-bold
  arial_geo-bold-italic
  arial_geo-italic
  bpg_algeti
  bpg_algeti_compact
  bpg_arial_2009
  bpg_boxo
  bpg_boxo-boxo
  bpg_classic_medium
  bpg_dedaena
  bpg_dedaena_nonblock
  bpg_excelsior_caps_dejavu_2010
  bpg_excelsior_dejavu_2010
  bpg_extrasquare_2009
  bpg_extrasquare_mtavruli_2009
  bpg_glaho
  bpg_glaho_2008
  bpg_glaho_arial
  bpg_glaho_bold
  bpg_glaho_sylfaen
  bpg_glaho_traditional
  bpg_ingiri_2008
  bpg_irubaqidze
  bpg_mrgvlovani_caps_2010
  bpg_nino_elite_exp
  bpg_nino_elite_ultra
  bpg_nino_elite_ultra_caps
  bpg_nino_medium_caps
  bpg_nino_mtavruli_bold
  bpg_nino_mtavruli_book
  bpg_nino_mtavruli_normal
  bpg_no9
  bpg_nostalgia
  bpg_paata
  bpg_paata_caps
  bpg_paata_cond
  bpg_paata_cond_caps
  bpg_paata_exp
  bpg_phone_sans_bold
  bpg_phone_sans_bold_italic
  bpg_phone_sa

# Explore data

In [9]:
import pandas as pd

In [11]:
df = pd.read_csv(data_dir/"metadata.csv")
print(df.head())
print(df.tail())

                        file_name          text
0  3d_unicode/3d_unicode_0000.png          ამათ
1  3d_unicode/3d_unicode_0001.png   პარტიანკაში
2  3d_unicode/3d_unicode_0002.png  კომენტარების
3  3d_unicode/3d_unicode_0003.png       ფრიდრიხ
4  3d_unicode/3d_unicode_0004.png   ცდწლოოწნწში
                                         file_name      text
100495  NotoSansGeorgian/NotoSansGeorgian_1495.png     რიგში
100496  NotoSansGeorgian/NotoSansGeorgian_1496.png     ალიკა
100497  NotoSansGeorgian/NotoSansGeorgian_1497.png      ტარს
100498  NotoSansGeorgian/NotoSansGeorgian_1498.png     კარგი
100499  NotoSansGeorgian/NotoSansGeorgian_1499.png  სასახლეს


In [12]:
print(df["text"].value_counts())

text
და              4221
არ              1163
რომ              995
იყო              768
კი               604
                ... 
ფუფთწჟჯკდპბგ       1
ოოწლღდჩქთტტ        1
გითქვამს           1
სიკვდილია          1
ბგშჩდეკშჰ          1
Name: count, Length: 38003, dtype: int64


In [13]:
# Check text length variations
df["text_len"] = df["text"].str.len()
print(df["text_len"].describe())

count    100500.000000
mean          6.390886
std           2.970310
min           2.000000
25%           4.000000
50%           6.000000
75%           8.000000
max          24.000000
Name: text_len, dtype: float64


# Prepare images

In [14]:
from transformers import TrOCRProcessor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")

# Test Georgian tokenization
test_text = "გამარჯობა"
tokens = processor.tokenizer.tokenize(test_text)
print(tokens)  # If you see lots of <unk> or weird splits, you need a custom tokenizer

E:\Mega\Python\Ka-OCR\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\iskhi\.cache\huggingface\hub\models--microsoft--trocr-base-handwritten. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fas

['á', 'ĥ', 'Ĵ', 'á', 'ĥ', 'Ĳ', 'á', 'ĥ', 'Ľ', 'á', 'ĥ', 'Ĳ', 'á', 'ĥ', 'ł', 'á', 'ĥ', '¯', 'á', 'ĥ', 'Ŀ', 'á', 'ĥ', 'ĳ', 'á', 'ĥ', 'Ĳ']


In [15]:
class GeorgianTokenizer:
    def __init__(self, max_length: int = 32):
        # Special tokens
        self.pad_token = "<pad>"
        self.bos_token = "<s>"      # beginning of sequence
        self.eos_token = "</s>"     # end of sequence
        self.unk_token = "<unk>"    # unknown character

        # Georgian alphabet (33 letters)
        self.georgian_chars = "აბგდევზთიკლმნოპჟრსტუფქღყშჩცძწჭხჯჰ"

        # Build vocabulary: special tokens + Georgian characters
        self.vocab = [self.pad_token, self.bos_token, self.eos_token, self.unk_token]
        self.vocab.extend(list(self.georgian_chars))

        # Create mappings
        self.char_to_id = {char: idx for idx, char in enumerate(self.vocab)}
        self.id_to_char = {idx: char for idx, char in enumerate(self.vocab)}

        # Token IDs for special tokens
        self.pad_token_id = 0
        self.bos_token_id = 1
        self.eos_token_id = 2
        self.unk_token_id = 3

        self.max_length = max_length

    def encode(self, text: str, padding: bool = True) -> list[int]:
        """Convert Georgian text to token IDs."""
        # Start with BOS token
        ids = [self.bos_token_id]

        # Convert each character
        for char in text:
            ids.append(self.char_to_id.get(char, self.unk_token_id))

        # Add EOS token
        ids.append(self.eos_token_id)

        # Truncate if too long
        if len(ids) > self.max_length:
            ids = ids[:self.max_length - 1] + [self.eos_token_id]

        # Pad if needed
        if padding:
            ids.extend([self.pad_token_id] * (self.max_length - len(ids)))

        return ids

    def decode(self, ids: list[int]) -> str:
        """Convert token IDs back to text."""
        chars = []
        for id in ids:
            if id in (self.pad_token_id, self.bos_token_id, self.eos_token_id):
                continue
            chars.append(self.id_to_char.get(id, ""))
        return "".join(chars)

    def __len__(self):
        return len(self.vocab)

In [16]:
tokenizer = GeorgianTokenizer(max_length=32)

# Test encoding
text = "გამარჯობა"
ids = tokenizer.encode(text)
print(f"Text: {text}")
print(f"IDs: {ids[:15]}...")  # First 15 tokens
print(f"Length: {len(ids)}")

# Test decoding
decoded = tokenizer.decode(ids)
print(f"Decoded: {decoded}")

# Verify vocab size
print(f"Vocab size: {len(tokenizer)}")  # Should be 37 (4 special + 33 Georgian)

Text: გამარჯობა
IDs: [1, 6, 4, 15, 4, 20, 35, 17, 5, 4, 2, 0, 0, 0, 0]...
Length: 32
Decoded: გამარჯობა
Vocab size: 37


In [17]:
import torch
from torch.utils.data import Dataset
from PIL import Image, ImageOps


class GeorgianOCRDataset(Dataset):
    def __init__(self, df: pd.DataFrame, root_dir: str, processor, tokenizer: GeorgianTokenizer):
        self.df = df
        self.root_dir = root_dir
        self.processor = processor
        self.tokenizer = tokenizer  # custom tokenizer

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int) -> dict[str, torch.Tensor]:
        text = self.df.iloc[idx]['text']
        file_path = f"{self.root_dir}/{self.df.iloc[idx]['file_name']}"

        # Open and process image
        img = Image.open(file_path).convert("RGB")
        w, h = img.size
        target_size = 384

        # Scale height to target_size, width proportionally
        scale = target_size / max(w, h)
        new_w, new_h = int(w * scale), int(h * scale)
        img = img.resize((new_w, new_h), Image.Resampling.BILINEAR)

        # Pad to square
        new_img = Image.new("RGB", (target_size, target_size), (255, 255, 255))
        offset = ((target_size - new_w) // 2, (target_size - new_h) // 2)
        new_img.paste(img, offset)

        # Use Processor for Normalization
        pixel_values = self.processor(new_img, return_tensors="pt").pixel_values

        # Tokenize Georgian Text
        labels = self.tokenizer.encode(text)

        # Replace padding token id with -100 so it's ignored by the loss function
        labels = [label if label != self.tokenizer.pad_token_id else -100 for label in labels]

        return {
            "pixel_values": pixel_values.squeeze(),
            "labels": torch.tensor(labels)
        }


In [19]:
from transformers import VisionEncoderDecoderModel, TrOCRProcessor

# Load processor (for image processing only)
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")

# Create your tokenizer
tokenizer = GeorgianTokenizer(max_length=32)

# Load model and resize token embeddings
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")
model.decoder.resize_token_embeddings(len(tokenizer))  # Resize to 37

# Configure special tokens
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

E:\Mega\Python\Ka-OCR\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\iskhi\.cache\huggingface\hub\models--microsoft--trocr-base-printed. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP do

# Train test split